Pipeline Errors for night of {{ params.day_obs }}
=====

In [ ]:
day_obs = "2024-02-29"
instrument = "LATISS"

In [ ]:
import lsst.daf.butler as dafButler
import tabulate
from dataclasses import dataclass

@dataclass
class error_summary:
    visit: int
    detector: int
    error_messages: list

butler_nocollection = dafButler.Butler("/repo/embargo")

collections = list(butler_nocollection.registry.queryCollections(f"LATISS/prompt/output-{day_obs:s}*"))
collection = collections[0]

b = dafButler.Butler("/repo/embargo", collections=collection)

In [ ]:
log_visit_detector = set([(x.dataId['exposure'], x.dataId['detector']) for x in b.registry.queryDatasets("isr_log")])
print("Number of ISRs attempted: {:d}".format(len(log_visit_detector)))

calexp_visit_detector = set([(x.dataId['visit'], x.dataId['detector']) for x in b.registry.queryDatasets("calexp")])
print("Number of successful calexps attempted: {:d}".format(len(calexp_visit_detector)))

missing_calexps = set(log_visit_detector - calexp_visit_detector)
missing_visits = [x[0] for x in missing_calexps]
print("Number of unsuccessful processCcd attempts (no resulting calexp): {:d}".format(len(missing_calexps)))

In [ ]:

log_dataset_types_exposure = ["isr_log"]
log_dataset_types_visit = ["characterizeImage_log", "calibrate_log"]

error_summaries = []
for visit, detector in missing_calexps:

    visit_errors = []
    
    for ds_types in log_dataset_types_exposure:
        log_messages = b.get(ds_types, dataId={"instrument": instrument, "exposure": visit, "detector": detector})
        isr_errors = [msg for msg in log_messages if msg.levelno > 30]
        visit_errors.extend(isr_errors)
    
    for ds_types in log_dataset_types_visit:
        log_messages = b.get(ds_types, dataId={"instrument": instrument, "visit": visit, "detector": detector})
        errors = [msg for msg in log_messages if msg.levelno > 30]
        visit_errors.extend(errors)

    error_summaries.append(error_summary(visit=visit, detector=detector, error_messages=visit_errors))



In [ ]:
def make_url_from_visit(visit):
    s = str(visit)
    day_string = f"{s[0:4]}-{s[4:6]}-{s[6:8]}"
    counter = int(s[8:])
    url = f"https://roundtable.lsst.codes/rubintv/summit/auxtel/monitor/event/{day_string:s}/{counter:d}"
    return url
    

table_contents = [(e.visit, e.detector, "<a href=\"" + make_url_from_visit(e.visit) + "\" target=\"_blank\">img</a>", e.error_messages[0].message) for e in error_summaries]
table = tabulate.tabulate(table_contents, tablefmt='unsafehtml', headers=("Visit", "Det", "Img", "Error Message"))
table